### Домашняя работа

**Задание простого уровня** Мы говорили, что метрики качества нужны, чтобы сравнивать различные модели между собой. В задаче полиномиальной регрессии реализуйте код для выбора лучшей степени полиному:

* возьмите все степени от 1 до 10 по порядку, без пропусков.
* найдите степень полинома, где будет лучший r2-score
* напишите код, который выводит самую подходящую степень полинома и соответствующий ей скор

Эта процедура называется Grid Search и помогает найти лучшие параметры для модели.

Обучите лучшую модель и сделайте predict

In [113]:
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

df = pd.read_csv('3.10_non_linear.csv')

columns = ['x_train']

for p in range(2, 11):
    column_name = f'x_train^{p}'
    df[column_name] = df['x_train'] ** p
    columns.append(column_name)

df.head()

,x_train,y_train,x_train^2,x_train^3,x_train^4,x_train^5,x_train^6,x_train^7,x_train^8,x_train^9,x_train^10
0,0.138368,0.838812,0.019146,0.002649,0.000367,0.000051,0.000007,9.710899e-07,1.343682e-07,1.859232e-08,2.572590e-09
1,0.157237,0.889313,0.024723,0.003887,0.000611,0.000096,0.000015,2.376188e-06,3.736243e-07,5.874751e-08,9.237275e-09
2,0.188684,1.430040,0.035602,0.006717,0.001267,0.000239,0.000045,8.514310e-06,1.606516e-06,3.031243e-07,5.719477e-08
3,0.685553,1.717309,0.469983,0.322198,0.220884,0.151427,0.103811,7.116823e-02,4.878957e-02,3.344783e-02,2.293025e-02
4,0.874237,2.032588,0.764290,0.668171,0.584140,0.510677,0.446452,3.903051e-01,3.412192e-01,2.983064e-01,2.607905e-01


In [114]:
y = df['y_train']

for i in range(1, 11):    
    X = df[columns].iloc[:, :i]
    reg = LinearRegression().fit(X, y)
    print(f'Степень полинома: {i}, r2 score: {r2_score(y,reg.predict(X))}')

Степень полинома: 1, r2 score: 0.5194693682784115
Степень полинома: 2, r2 score: 0.5334743428787219
Степень полинома: 3, r2 score: 0.8884043369162163
Степень полинома: 4, r2 score: 0.8954938957170193
Степень полинома: 5, r2 score: 0.8955568441439694
Степень полинома: 6, r2 score: 0.8993403708180607
Степень полинома: 7, r2 score: 0.90046097540244
Степень полинома: 8, r2 score: 0.9086556247250233
Степень полинома: 9, r2 score: 0.9088071215143878
Степень полинома: 10, r2 score: 0.9091133831292173


**Задание среднего уровня** Напишите класс для обучения модели, который содержит:

* функцию `.fit(X, y)` , которая принимает на вход массив фичей `X`, массив таргетов `y` и обучает коэффициенты регрессии. Код для обучения взять из первого урока модуля *Постановка ML задачи линейной регрессии*
* функцию `.predict(X)`, которая по массиву фичей `X` возвращает массив предсказаний `y`

Нужно использовать код для аналитически вычисляемых коэффициентов. 

Это задание позволит понять, как работает линейная регрессия "внутри" библиотечной реализации.

In [128]:
import numpy as np

class CustomLinearReg:
    def __init__(self):
        pass
    
    def fit(self, X, y):

        # добавление единичного столбца ухудшает модель
        # columns = list(X.columns)
        # X['bias'] = 1
        # X = X[['bias'] + columns]

        self.weights = np.linalg.inv(
            (X.T).dot(X)
        ).dot(
            X.T
        ).dot(y)
    
    def predict(self, X):
        return X.dot(self.weights)

a = CustomLinearReg()

# при степени полинома меньше 3 почему-то работает плохо
X = df[columns].iloc[:, :3]
y = df['y_train']

a.fit(X, y)
reg.fit(X, y)
print(f'CustomLinearReg\nКоэффициенты: {a.weights}, r2 score: {r2_score(y, a.predict(X))}')
print(f'\nLinearRegression\nКоэффициенты: {reg.coef_}, r2 score: {r2_score(y, reg.predict(X))}')


CustomLinearReg
Коэффициенты: [ 2.95231629 -1.2779185   0.13665884], r2 score: 0.8350917275477862

LinearRegression
Коэффициенты: [ 2.00644167 -0.98019974  0.10894323], r2 score: 0.8884043369162163


**Задание высокого уровня**

1. разделите датасет с домами Бостона из Урока 2 (таргет и фичи) на две части: в одной части 80% датасета (назовём train) в другой 20% (назовём valid) с помощью функции `train_test_split` из библиотеки `sklearn`
1. обучите модель только на train датасете
1. постройте предсказания valid датасете
1. Посчитайте  `r2 score` на валидационном сете

После этого примените к обеим датасетам z-преобразование и повторите шаги 2-4. Как изменилась метрика r2?

Это задание поможет понять, как валидировать линейную регрессию (и другие модели) на отложенной выборке.

In [184]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)

for i in range(9):

    # До стандартизиованной оценки
    data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
    target = raw_df.values[1::2, 2]

    X_train, X_valid, y_train, y_valid = train_test_split(data, target, test_size=0.2, random_state=i)
    reg = LinearRegression().fit(X_train, y_train)
    print(i, r2_score(y_valid, reg.predict(X_valid)))

    # После стандартизированной оценки 

    data = StandardScaler().fit_transform(data)
    target = StandardScaler().fit_transform(target.reshape(-1, 1))

    X_train, X_valid, y_train, y_valid = train_test_split(data, target, test_size=0.2, random_state=i)

    reg = LinearRegression().fit(X_train, y_train)

    print(i, r2_score(y_valid, reg.predict(X_valid)), '\n')

0 0.5892223849182514
0 0.5892223849182505 

1 0.763417443213847
1 0.7634174432138471 

2 0.7789207451814423
2 0.7789207451814418 

3 0.7952617563243876
3 0.7952617563243857 

4 0.7263451459702503
4 0.7263451459702509 

5 0.7334492147453111
5 0.7334492147453071 

6 0.6839026890069033
6 0.6839026890069029 

7 0.5785415472763411
7 0.5785415472763407 

8 0.707962796713453
8 0.7079627967134524 



Похоже, для такой модели приведение данных к стандартизированной оценке не имеет смысла. Верноятно, вместо масштабированния данных с тем же успехом можно масштабировать коэффициенты линейной регрессии, что и происходит при применении модели.